In [29]:
import pandas as pd
import numpy as np

In [30]:
flavors = pd.read_csv('flavors.csv')

In [31]:
flavors = flavors.dropna()
flavors = flavors.rename(columns={"celery":"ingredient", "vegetable":"flavor"})

In [32]:
sweet = pd.DataFrame([['simple','sweet'],['agave','sweet'],['sugar','sweet'],['demerara','sweet'],['honey','sweet'],['maraschino','sweet'],["vanilla","sweet"]], columns=['ingredient','flavor'])
bitter = pd.DataFrame([['bitter','bitter'],['bitters','bitter'],['amara','bitter'],['coffee','bitter'],['cacao','bitter'],['americano','bitter']], columns=['ingredient','flavor'])
salty = pd.DataFrame([['salt','salty'],['tomato','salty'],['olive','salty'],['pickle','salty'],['cheese','salty']], columns=['ingredient','flavor'])
herbaceous = pd.DataFrame([['mint','herbaceous'],['basil','herbaceous'],['thyme','herbaceous'],['pickle','salty'],['cheese','salty']], columns=['ingredient','flavor'])
add = [sweet, bitter, salty, herbaceous]
add_name = ['sweet', 'bitter', 'salty']

In [33]:
cocktail_flavors = ['floral','citrus','herbaceous','fruity']

ingredient_flavors = pd.DataFrame()
for f in cocktail_flavors:
    ingredient_flavors = ingredient_flavors.append(flavors[flavors.flavor==f])

In [34]:
for i in add:
    ingredient_flavors = ingredient_flavors.append(i, ignore_index=True)

In [35]:
# creating dictionary of ingredients: flavors
temp = ingredient_flavors.set_index(ingredient_flavors['ingredient']).drop(['ingredient'], axis=1)
ingredients_dict = temp.to_dict()['flavor']

In [36]:
ingredients_dict = dict(sorted(ingredients_dict.items(), key=lambda item: item[1]))

In [37]:
# list of flavors
flavors_list = sorted(ingredient_flavors['flavor'].unique())
n_flavors = len(flavors_list)

In [38]:
# cocktail csv
cocktails_csv = pd.read_csv('cocktail_flavors_ingreds_combined_denumbered.csv')
cocktails_csv['Ingredients'] = cocktails_csv['Ingredients'].apply(str.lower)

In [39]:
# mask of which cocktails have certain ingredients
mask = [cocktails_csv['Ingredients'].str.lower().str.contains(ingr) for ingr in ingredients_dict.keys()]
keys = list(ingredients_dict.keys())

In [40]:
keys.index("vanilla")

68

In [41]:
cocktails_csv['Ingredients'][546]

'luxardo amaro abano, luxardo amaretto di saschira, espresso coffee, 1 scoop vanilla ice cream'

In [42]:
mask[68][546]

True

In [43]:
st = [0]
end = []
for k in range(len(keys)-1):
    if ingredients_dict[keys[k]] != ingredients_dict[keys[k+1]]:
        end += [k+1]
        st += [k+1]
end += [len(keys)]

In [44]:
flavors_list

['bitter', 'citrus', 'floral', 'fruity', 'herbaceous', 'salty', 'sweet']

In [45]:
end

[6, 11, 25, 49, 57, 62, 69]

In [46]:
len(keys)

69

In [47]:
merged_mask = np.zeros((n_flavors,616))
for i, s, e in zip(range(len(flavors_list)), st, end):
    merged_mask[i] = mask[s] #initializing merge to the first mask 
    for j in range(s+1, e):
        if j == 68:
            print(merged_mask[i][546])
            print(mask[j][546])
            print(np.logical_or(merged_mask[i], mask[j])[546])
        merged_mask[i] = np.logical_or(merged_mask[i], mask[j]) #logical or merged mask and mask to get combined 

0.0
True
True


In [48]:
df = cocktails_csv.copy()
_map = {True:1, False:0}
i, _ = merged_mask.shape
for t in range(len(st)):
    df[flavors_list[t]] = merged_mask[t]

In [49]:
def get_flavors(row):
    flavors = []
    for col in flavors_list:
        if row[col] == 1:
            flavors.append(col)
    return ', '.join(flavors)

df['flavors'] = df.apply(get_flavors, axis=1)

In [50]:
df = df.drop(['spicy'], axis=1)

In [51]:
df.loc[df['Cocktail Name']=='Mojito']

,Cocktail Name,Glass,strIBA,Ingredients,Instructions,floral,citrus,herbaceous,fruity,sweet,bitter,combined,flavors,salty
344,Mojito,Highball glass,Contemporary Classics,"light rum,lime,sugar,mint,soda water",Muddle mint leaves with sugar and lime juice. ...,0.0,1.0,1.0,0.0,1.0,0.0,"light rum,lime,sugar,mint,soda water,citrus,sweet","citrus, herbaceous, sweet",0.0


In [52]:
df.to_csv('updated_cocktail_flavors_ingreds_combined_denumbered.csv')